In [ ]:
#from core.utils import Tibanna
#from core import ff_utils
from uuid import UUID
import ast
import json
import copy
from wranglertools import fdnDCIC
#env = 'fourfront-webprod'
#tibanna = Tibanna(env=env)
#ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
#ff = ff_utils.fdn_connection(key={"key": "63A2CVAK", "secret": "sdlxhiazfux6aki2", "server": "http://localhost:8000"})
def connect2ff():
    # import pdb; pdb.set_trace()
    key = fdnDCIC.FDN_Key({'blah':{"key": "DVP6M4DG", "secret": "xf2dqzetsjovmbhk", "server": "https://data.4dnucleome.org"}}, 'blah')
    conn = fdnDCIC.FDN_Connection(key)
    return conn

ff = connect2ff()

"""Does the string look like a uuid"""
def is_uuid(value):
    if '-' not in value:
        #md5checksums are valid uuids but do not contain dashes so this skips those
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False


"""Find any uuids in the value"""
def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals          


"""Given an ID for an item all descendant linked item uuids (as given in 'frame=raw')
    are stored in a dict with each item type as the value.
    All descendants are retrieved recursively except the children of the types indicated
    in the no_children argument.
    The relationships between descendant linked items are not preserved - i.e. you don't
    know who are children, grandchildren, great grandchildren ..."""
def get_linked_items(itemid, found_items, no_children=['Publication']):
    # import pdb; pdb.set_trace()
    if not found_items.get(itemid):
        res = fdnDCIC.get_FDN(itemid, connection=ff, frame='raw')
        if 'error' not in res['status']:
            # create an entry for this item in found_items
            try:
                obj_type = fdnDCIC.get_FDN(itemid, connection=ff)['@type'][0]
                found_items[itemid] = obj_type
            except:
                print "Can't find a type for item %s" % itemid
            if obj_type not in no_children:
                fields_to_check = copy.deepcopy(res)
                id_list = []
                for key, val in fields_to_check.iteritems():
                    # could be more than one item in a value
                    foundids = find_uuids(val)
                    if foundids:
                        id_list.extend(foundids)
                if id_list:
                    id_list = [i for i in list(set(id_list)) if i not in found_items]
                    for uid in id_list:
                        found_items.update(get_linked_items(uid, found_items))
    return found_items


"""Will filter items from a dictionary based on values
    can be either an inclusive or exclusive filter
    if include=False will remove the items with given values
    else will remove items that don't match the given values
"""
def filter_dict_by_value(dictionary, values, include=True):
    if include:
        return {k:v for k, v in dictionary.iteritems() if v in values}
    else:
        return {k:v for k, v in dictionary.iteritems() if v not in values}


"""Returns True if the field is present in the item
    BUT if there is value parameter only returns True if value provided is
    the field value or one of the values if the field is an array
    How fancy do we want to make this?"""
def has_field_value(item_dict, field, value=None, val_is_item=False):
    # 2 simple cases
    if not item_dict.get(field):
        return False
    if not value and item_dict.get(field):
        return True
    
    # now checking value
    val_in_item = item_dict.get(field)
    
    if isinstance(val_in_item, list):
        if value in val_in_item:
            return True
    elif isinstance(val_in_item, basestring):
        if value == val_in_item:
            return True

    # only check dict val_is_item param is True and only 
    # check @id and link_id - uuid raw format will have been
    # checked above
    if val_in_item:
        if isinstance(val_in_item, dict):
            ids = [val_in_item.get('@id'), val_in_item.get('link_id')]
            if value in ids:
                return True
    return False
            

"""find items that have the passed in fieldname in their properties
    even if there is currently no value for that field"""
def get_types_that_can_have_field(field):
    profiles = fdnDCIC.get_FDN('/profiles/', connection=ff, frame = 'raw')
    types_w_field = []
    for t, j in profiles.iteritems():
        if j['properties'].get(field):
            types_w_field.append(t)
    return types_w_field

    
"""Creates a joint analysis tag - creating a tags field if item doesn't already
    have one or append ja tag if it does"""
def make_ja_tag_patch(res):
    # import pdb; pdb.set_trace()
    tag = 'Joint Analysis 2018'
    if res.get('tags'):
        tags = res['tags'].append(tag)
    else:
        tags = [tag]
    return {'tags': tags}


def main():
    # items = ["1324d727-2ac8-4622-8ad3-b6e209235087", "1324d727-2ac8-4622-8ad3-b6e209235087", "blah", "e58e2141-9253-4c91-85c3-d67ce06db28f"]
    # items = ["431106bc-8535-4448-903e-854af460b260"]
    types2exclude = ['OntologyTerm', 'Ontology','Lab', 'User', 'Award', 'ExperimentSetReplicate']
    releasable = get_types_that_can_have_field('public_release')
    #search = '/search/?status=released&type=Biosource'
    #search = '/search/?type=ExperimentSetReplicate&experimentset_type=replicate&experiments_in_set.experiment_type=repliseq&status=released'
    search = '/search/?date_released%21=No%20value&experimentset_type=replicate&lab.title=Erez%20Lieberman%20Aiden%2C%20BCM&lab.title=Job%20Dekker%2C%20UMMS&lab.title=Bing%20Ren%2C%20UCSD&lab.title=Rene%20Maehr%2C%20UMMS&status=released&type=ExperimentSetReplicate'
    res = fdnDCIC.get_FDN(search, ff)
   
    bsuuids = []

    for item in res['@graph']:
        bsuuids.append(item['uuid'])
     
    seen = []
    count = 0
    for item in res['@graph']:
        uid = item['uuid']
        reldate = item['date_released']
        count = count + 1
        print '\nWorking on ', uid, count
        linked = {}
        linked = get_linked_items(uid, linked)
        flinked = filter_dict_by_value(linked, types2exclude, include=False)
        for i, t in flinked.iteritems():
            if i not in seen:
                print i, t
                seen.append(i)
                # import pdb; pdb.set_trace()
                item = fdnDCIC.get_FDN(i, ff)
                if 'public_release' not in item or 'date_released' not in item:
                    if t in releasable:
                        patch = {'public_release': reldate}
                        dry_run = False
                        if not dry_run:
                            pres = fdnDCIC.patch_FDN(i, ff, patch)
                            print pres
                            print "--SUCCESS!"
                        print "Adding Public Release Date %s to %s of type %s" % (reldate, i, t)
 
                    
    
    
    #taggable = get_types_that_can_have_field('tags')
    
    
    #for it in items:
    #    linked = get_linked_items(it, linked)
        
    # more efficient to do this on the full dict rather than item by item
    #items2tag = filter_dict_by_value(linked, taggable)
    
    #for it, ty in items2tag.iteritems():
        # import pdb; pdb.set_trace()
        #res = fdnDCIC.get_FDN(it, ff)
        #if not has_field_value(res, 'tags', 'Joint Analysis'):
            # patch the item with the tag
            #patch = make_ja_tag_patch(res)
            #dry_run = True
            #if not dry_run:
                #patch = fdnDCIC.patch_FDN(it, ff, patch)
                #print patch
                #print "--SUCCESS!"
            #print "Adding JA tag to %s of type %s" % (it, ty)
            
    
    #linked = filter_dict_by_value(linked, exclude, include=False)
    
    #for l, t in linked.iteritems():
    #    print 'GOT IT: ', l, '\t', t
 
 

        
if __name__ == '__main__':
    main()